## Installing Psi4


- Installation:
    - Conda installation is preferred:
    ```bash
    conda create -n <envname> psi4 psi4-rt -c psi4/label/dev
    ```
    - Download a pre-packaged binary from [psicode.org](http://psicode.org)
    - Compile from source. Not advised.

- Documentation:
    - Docs of `master` version linked to [psicode.org/psi4manual/master/](https://psicode.org/psi4manual/master/)
    - Forums on [forum.psicode.org](http://forum.psicode.org)
    - Development support & **bug reports** on github: [github.com/psi4/psi4](https://github.com/psi4/psi4/)

## Running Psi4

Three main ways to run Psi4:
  - as an executable (Psithon)
  - as a python module (Psi API)
  - via QCSchema / QCEngine

<img src="images/psi4_schema.jpeg"/>

### Psithon

- standalone input and output file
- no need for python knowledge
- most analogous with Gaussian, Orca etc.


- job specification within job script:
  - memory using `set memory 4GB`
  - number of cores using `set num_threads 2`

- command line options:
  - `-i` input file, by default `input.dat`
  - `-o` output file, by default `output.dat`, or if `-i` is specified `<input>.out`
  - `-n` to specify number of threads
  - `-l` to override Psi4's data dir (mainly basis set data)
  - `-s` to override Psi4's scratch dir, by default `/tmp/`
  - `-v` verbose output

- useful environmental variables:
  - `PATH` for `psi4`, `dftd3`, `gcp` binaries, conda should take care of this
  - `PSI_SCRATCH` for Psi4's scratch dir, by default `/tmp/`

### Psithon example: 

#### FeC triplet with HF/cc-pVDZ-PP

- using direct SCF (as opposed to the default DF)
- tight convergence criteria.

---
```python
molecule dimer {
 0 3
 Fe 0 0 0
 C  0 0 3.1558426295145727
}

set reference rohf
set scf_type direct
set e_convergence 1e-11
set r_convergence 1e-11
set d_convergence 1e-11

energy('hf/cc-pvdz-pp')
```
---

More details at https://psicode.org/psi4manual/master/tutorial.html

### Psi API

- use psi4 following `import psi4` in python
- commands normally behind `psi4` namespace
- can do more complex things with wavefunction objects, loops, etc.
- **Jupyter compatible!**

- options may be set using a single dictionary:
```python
psi4.set_options({
    "e_convergence": 1e-11,
    "r_convergence": 1e-11,
    "reference": 'rohf',
    "freeze_core": True
})
```

- accessing and storing outputs is tricky:
  - printed into `stdout`
  - possible to redirect using 
```python
psi4.core.set_output_file(<str output.file>, <bool Append?>)
```
- respects environmental variables such as `PSI_SCRATCH`, `PATH`, `PYTHONPATH`

### Psi API example: 
#### Running geometry optimizations with preset parameters
The following Python script: 
- runs a geometry optimisation as:
```bash
python job.py --functional pbe --basis pcseg-2 --nthreads 4 --memory 16 input.xyz
```
- dumps the optimized geometry into a file
- the output filename is a SHA256 of `pbe/pcseg-2:input.xyz`

### Psi API example: 
#### Imports
---

```python
import psi4
import json
import argparse
import os
import sys
import hashlib
```

### Psi API example: 
#### Input parsing with `argparse`
---
```python
parser = argparse.ArgumentParser(
    description="Run a geometry optimization starting with an xyz file and a method")
parser.add_argument("xyzfile", help = "path to input XYZ file")
parser.add_argument("--functional", help = "method to use",
                    default="b3lyp-d3bj", type = str.lower)
parser.add_argument('--basis', help="basis set to use",
                    default="def2-svpd", type = str.lower)
parser.add_argument('--nthreads', help="number of threads", default=4, type = int)
parser.add_argument('--memory', help="amount of memory (GB)", default=4, type = float)
parser.add_argument('--outpath', help="path to putput folder", default="results")
parser.add_argument('--cluster', default="local", type = str.lower)
parser.add_argument('--hess', default=-1, type = int)
parser.add_argument('--dyn', default=0, type = int)
args = parser.parse_args()
````

### Psi API example:
#### Some sanity checks to avoid duplication
---
```python
assert os.path.exists(args.xyzfile)
infolder, xyzfile = os.path.split(args.xyzfile)
method = f"{args.functional}/{args.basis}"
outmoltag = f"{method}:{xyzfile}"
outmolhash = hashlib.sha256(outmoltag.encode("utf-8")).hexdigest()[:10]
outmolfn = f"{outmolhash}.xyz"

if os.path.exists(os.path.join(args.outpath, outmolfn)):
    print(f"Job {outmoltag} already performed. Skipping.")
    sys.exit()
```

### Psi API example: 
#### Psi4 inputs are set here
---
```python
print(f"Calculating job {outmoltag} now.")
with open(os.path.join(infolder, xyzfile), "r") as infile:
    inmolstr = infile.read().strip()
inmol = psi4.geometry(inmolstr)

psi4.set_memory(f"{args.memory:4f}GB")
psi4.set_num_threads(args.nthreads)
psi4.set_options({
    "dft_spherical_points": 974,
    "dft_radial_points": 150,
    "g_convergence": "interfrag_tight",
    "full_hess_every": args.hess,
    "dynamic_level": args.dyn
})
```

### Psi API example: 
#### Calculate.
```python
E, wfn = psi4.optimize(method, molecule = inmol, return_wfn = True)
```

### Psi API example: 
####  And save.
---
```python
outmolarr = wfn.molecule().save_string_xyz_file().split("\n")
outmolarr[1] = outmoltag
os.makedirs(args.outpath, exist_ok = True)
with open(os.path.join(args.outpath, f"{outmolhash}.xyz"), "w") as outfile:
    outfile.write("\n".join(outmolarr))
```
---

More information at https://psicode.org/psi4manual/master/psiapi.html

### Via QCSchema + QCEngine / QCFractal

- either as 

    `psi4 --schema <input.json>`
- or within python using

    `qcng.compute(input_dict, "psi4", ...)`

<img src="images/MolSSI-Logo-2.jpg"/>

#### What are QCSchema + QCEngine
- unified specification of QM inputs
- E, G, H calculations possible
- translation layer to many QM packages, including Psi4, PySCF, Q-Chem, etc...
- very good for production runs of many identical jobs
- outputs are specially formatted `json` files, including:
  - date, hostname, information about environment
  - information about QM package used
  - a copy of input json
  - full output file (optional)
  - a dictionary of calculated quantities
- Peter's gold standard <sup>TM</sup> in reproducible computational data

More info about the schema: https://molssi-qc-schema.readthedocs.io/en/latest/index.html

More info about the engine: https://qcengine.readthedocs.io/en/latest/

#### What is QCFractal
- a way to handle your many clusters in a smart, transparent way
  - job submission on `gridengine`, `pbs`, `slurm`...
  - once set-up, don't have to worry about writing inputs
- a database of all calculations performed
  - a newly submitted job gets queried against the database first
  - easy plotting, benchmarking, etc.
- good if you have your own always-on fileserver
- probably an overkill for most cases

More info on qcfractal: http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/

### QCEngine example:
#### Running single point calculations using schema
The following python script:
  - runs a single point calculation
  - loads structural and computational data from a `json` file
  - modifies the `method` and `basis` from arguments
  - sets `memory` and `ncores` from arguments
  - writes out the `json` if successful
  - cleans up inputs if asked to

### QCEngine example:
#### Imports. Note `qcengine` is imported.
---
```python
import psi4
import os
import json
import qcengine as qcng
import argparse
```

### QCEngine example:
#### Argument parsing with `argparse`, again.
---
```python
parser = argparse.ArgumentParser(description="Run single point energies of an json schema files")

parser.add_argument('--input', help="path to JSON schema file")
parser.add_argument('--output', help="requested output path")
parser.add_argument('--functional', help="method to use", required = True, type = str.lower)
parser.add_argument('--basis', help="basis set to use", required = True, type = str.lower)
parser.add_argument('--nthreads', help="number of threads", default=4, type = int)
parser.add_argument('--memory', help="amount of memory", default=4, type = float)
parser.add_argument('--clean', help="file prefix to clean", default = False)
parser.add_argument('--tag', default="")

args = parser.parse_args()
```

### QCEngine example:
#### Load input `json` and augment `"model"` key: 
---
```python
assert os.path.exists(args.input)
with open(args.input, "r") as infile:
    molschema = json.load(infile)
molschema["model"] = {
    "method": args.functional,
    "basis": args.basis
}
```

```json
{
 "molecule": {
  "symbols": [
   "C",   "C"
  ],
  "geometry": [
   0.0,   0.0,   0.0,
   0.0,   0.0,   2.343260395567707
  ],
  "molecular_charge": 0,
  "molecular_multiplicity": 1
 },
 "driver": "energy",
 "model": {},
 "keywords": {
  "reference": "RKS",
  "dft_spherical_points": 974,
  "dft_radial_points": 150,
  "guess": "SAP"
 }
}
```

### QCEngine example:
#### Execute our `molschema` with `psi4` via `qcengine`:
---
```python
ret = qcng.compute(molschema, "psi4",
                    return_dict=True,
                    local_options={"memory": args.memory, "ncores": args.nthreads})
```

### QCEngine example:
#### Output handling and cleaning:
---
```python
if ret["success"]:
    ret["tag"] = args.tag
    with open(args.output, "w") as outfile:
        json.dump(ret, outfile, indent=1)
    if args.clean:
        cleanlist = [fn for fn in os.listdir(os.getcwd()) if fn.startswith(args.clean)]
        for fn in cleanlist:
            os.remove(fn)
else:
    print(ret)
    raise RuntimeError
```

## Basics of Psi4 calculations

- molecule specification
- single point energies
- optimizations (gradients)
- vibrational analysis (Hessians)

### Molecule specification:

#### Cartesians in Å
---
```python
molecule FeC {
 0 3
 Fe 0 0 0
 C  0 0 3.1558426295145727
}
```

#### Z-matrix in Bohr
---
```python
molecule FeC {
 0 3
 Fe 
 C  6
 units bohr
}
```

#### Or from an XYZ file (Psi API syntax):
---
```python
with open("input.xyz", "r") as infile:
    inmolstr = infile.read().strip()
FeC = psi4.geometry(inmolstr)
FeC.set_multiplicity(3)
```

#### Or maybe from `pubchem` (Psi API syntax):
---
```python
O2 = psi4.geometry("pubchem:O2")
O2.set_multiplicity(3)
```

### Molecule specification details:

- you can specify more than one molecule in one script: the argument before `{}` becomes its name

- you can split molecule into fragments using `--`:

    ```
    molecule XeXe {
      0 1
      Xe 0 0 0
      --
      0 1
      Xe 0 0 4.1
    }
    ```

- you can set ghost atoms using `@` or `(Gh)`:

    ```
    molecule XeXe {
      0 1
      Xe   0 0 0
      @Xe  0 0 2.05
      Xe   0 0 4.10
    }
    ```

- you can request lower (e.g. C$_1$) symmetry by appending `symmetry c1`
- commands `no_reorient` and `no_com` are useful to stop rotating and translating the molecule

### Molecule specification via templates:
- very powerful for PES searches:
  - first, define a template:
```python
template = """
           0 1
           He           X_POS Y_POS    0.000000000000
           --
           0 1
           C            0.000 0.000    0.000000000000
           O           -1.161 0.000    0.000000000000
           S            1.561 0.000    0.000000000000
           no_reorient
           no_com
           """
angles = [offset + each * 0.5 for each in range(0,nangles)]
radii  = [2.2 + each * 0.05 for each in range(0,61)]
```

### Molecule specification via templates:
- very powerful for PES searches:
  - second, modify the template and run a calculation
```python
for angle in angles:
    for radius in radii:
        y = math.sin(angle*(2*math.pi/360)) * radius
        x = math.cos(angle*(2*math.pi/360)) * radius * -1
        molecule = template.replace("X_POS", "{0:f}".format(x))
        molecule = molecule.replace("Y_POS", "{0:f}".format(y))
        print(molecule)
        geometry = psi4.geometry(molecule)
        geometry.set_molecular_charge(0)
        geometry.set_multiplicity(1)
        geometry.update_geometry()
        [...]
```

### Single point energies:

- `method/basis` syntax:
```python
energy('pbe0-d3/cc-pvdz', molecule = XeXe)
```

- separate basis syntax:
```python
set basis cc-pvdz
energy('pbe0-d3', molecule = XeXe)
```

- $\psi$ can be returned using `return_wfn = True`:
````python
E, wfn = energy(..., return_wfn = True)
````
- methods include: many DFAs, HF, MP2 through MP4, CCSD, CCSD(T), CISD, FCI, SAPT, ...
- basis set families include: cc-pvXz, aug-cc-pvXz, def2-Xzvp, def2-Xzvpp, def2-Xzvppd, pcseg-N, aug-pcseg-N, ...

### Single point energies: Module selection

- by default, density fitting is used for all methods: this **requires** a proper **auxilliary basis set**
- most included families (cc-pvXz, def2-Xzvp and augmented variants) have a sensible default up to 4-$\zeta$
- for exotic or large basis sets, use non-DF variants:
  - for SCF (and DFT): `set scf_type pk` works well
  - for post-HF methods: `set scf_type direct`, `set mp2_type conv`, `set cc_type conv` works well


### Single point energies: DFT grid selection

- the default integration grid is quite small - unpruned (75, 302) as of Apr. 2021
- change grid parameters to (99, 590) using:
    ```
    set dft_spherical_points 590
    set dft_radial_points 99
    ```
  the full list of available grids is [here](https://psicode.org/psi4manual/master/dft.html#grid-selection).
- grid point pruning can be turned on using:
    ```
    set dft_pruning_scheme robust
    ```

### Optimizations and Gradients

- Gradients:
  - analytical gradients available for most methods
  - **excluding** DHDFA and DFT-V or DFT-NL
  - findif can be done with anything
- Optimization:
  - handled by OPTKING, about to be replaced by py-optking

- `method/basis` syntax:
```
optimize('hf/cc-pvdz', molecule = XeXe)
```

- separate basis syntax:
```
set basis cc-pvdz
optimize('hf', molecule = XeXe)
```

- note that the `molecule` object passed to `optimize` is changed during the optimization
    - use `molecule = XeXe.clone()` if you want to keep the original unchanged

### Optimization convergence and conv. criteria

Criteria:
- reasonably tight [defaults](https://psicode.org/psi4manual/master/optking.html#convergence-criteria), equivalent to Q-chem
- possible to tighten using `set g_convergence gau_tight` or `gau_verytight` for Gaussian-equivalents
- for vdW complexes, `set g_convergence interfrag_tight` works well

Convergence:
- supplying an initial Hessian with `set full_hess_every 0` helps
- Hessian can be calculated with a cheaper method:
```python
optimize("ccsd(t)/aug-cc-pvtz", ..., hessian_with="mp2")
```
- for difficult cases, `set dynamic_level 1` or `set opt_coordinates both` may help

### Optimization types
- minimum is default, TS and IRC can be accessed with `set opt_type ts` or `set opt_type irc`
  - note that you _really_ want a good Hessian for TS or IRC search
- constrained optimization:
  - possible, but quite finicky with syntax:
```
   molecule {
     H
     O 1 0.90
     O 2 1.40 1 100.0
     H 3 0.90 2 100.0 1 90.0
   }
   set optking {
     frozen_dihedral = ("""
       1 2 3 4
     """)
   }
   optimize('scf')
```

### Gradient / Optimization caveats:

- ECP gradients are numerical - analytical ones coming in Psi4 1.5 via libecpint
- counterpoise-corrected optimization (or energy) possible with:
```python
optimize('m062x/def2-svpd', molecule = dimer, bsse_type = 'cp')
```
  this of course requires the `dimer` to be specified with two fragments.
- can also use the `cbs` wrapper in optimization.

### Frequencies and Hessians

- Hessians:
  - analytical gradients available only for HF
  - findif can be done with anything:
  - accessed through: 
```
H, wfn = hessian('blyp/pseg-0', [...], return_wfn = True)
```



- Frequencies:
  - accessible directly from `frequency('blyp/pcseg-0', [...])`
  - or indirectly using `freqs = vibanal_wfn(wfn)` if you have a Hessian already

### Thermochemical analysis
- printed by default if you request a `frequency([...])` calculation
  - 298.15 K, 101325 Pa, most common isotope masses
  - can be adjusted and re-ran at different T, p as follows:
```
H, wfn = frequency('hf3c', return_wfn = True)
set t 273.15
set p 100000
thermo(wfn, wfn.frequencies())
```

- more control can be achieved with `vibanal_wfn()`:
```python
G, wfnG = optimize("hf3c", return_wfn = True, molecule = h2o)
H, wfnH = frequency("hf3c", ref_gradient = wfnG.gradient(), return_wfn = True, molecule = h2o)
# Modify h2o -> d2o:
d2o = h2o.clone()
d2o.set_mass(0, 2.014101779)
d2o.set_mass(2, 2.014101779)
vibinfo = vibanal_wfn(wfnH, molecule = d2o)
```

 ### Thanks for listening!

[Session 2](session_2.ipynb)